In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os

# Definir transformações para os dados de validação
data_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def load_model(model_path):
    model = models.efficientnet_b0(pretrained=True)
    num_ftrs = model.classifier[1].in_features
    model.classifier = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(num_ftrs, 1)
    )
    model.load_state_dict(torch.load(model_path))
    model = model.to(device)
    return model

def evaluate_new_images(model, data_dir, transform):
    model.eval()
    new_dataset = datasets.ImageFolder(root=data_dir, transform=transform)
    dataloader = DataLoader(new_dataset, batch_size=1, shuffle=False)
    
    all_preds = []
    all_labels = []
    file_names = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device).float()
            outputs = model(inputs)
            preds = torch.sigmoid(outputs).round()
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            # Obter o nome do arquivo
            path = dataloader.dataset.samples[len(all_preds)-1][0]
            file_name = os.path.basename(path)
            file_names.append(file_name)
            # Exibir o resultado para cada imagem
            label_idx = int(preds.item())
            label_name = new_dataset.classes[label_idx]
            print(f'File: {file_name}, Predicted: {label_name}')

    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)

    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1 Score: {f1:.4f}')

    print('\nClassification Report:')
    print(classification_report(all_labels, all_preds, target_names=new_dataset.classes))

    # Plot da matriz de confusão
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=new_dataset.classes, yticklabels=new_dataset.classes)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()

# Caminho para o arquivo salvo do modelo
model_path = 'modelo_camisas_sport_nautico.pth'

# Carregar o modelo
model = load_model(model_path)

# Diretório contendo novas imagens para avaliação
new_data_dir = 'imagens/test'  # Atualize com o caminho do seu diretório de novas imagens

# Avaliar com novas imagens
evaluate_new_images(model, new_data_dir, data_transforms)
